In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Lambda,Bidirectional,LSTM,Embedding
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np



In [2]:
train_data=tfds.load('tiny_shakespeare',split=['train'])
val_data=tfds.load('tiny_shakespeare',split=['validation'])

In [3]:

for i in train_data[0]:
  train_data=i['text'].numpy().decode()
  
for i in val_data[0]:
  val_data=i['text'].numpy().decode()
  
  

In [4]:
tokenizer=Tokenizer(char_level=True)
tokenizer.fit_on_texts(train_data)
vocab_size=len(tokenizer.word_index)

In [5]:
[train_sentence]=np.asarray(tokenizer.texts_to_sequences([train_data]),np.float32)
[val_sentence]=np.asarray(tokenizer.texts_to_sequences([val_data]),np.float32)

In [6]:
window_size=10
shuffle_buffer_size=1000
batch_size=32
val_data=tf.data.Dataset.from_tensor_slices(val_sentence)
val_data=val_data.window(window_size,shift=1,drop_remainder=True)
val_data=val_data.flat_map(lambda x: x.batch(window_size+1))
val_data=val_data.shuffle(shuffle_buffer_size)
val_data=val_data.map(lambda x:(x[:-1],x[1:]))
val_data=val_data.batch(batch_size).prefetch(1)

In [7]:
window_size=10
shuffle_buffer_size=1000
batch_size=32
training_data=tf.data.Dataset.from_tensor_slices(train_sentence)
training_data=training_data.window(window_size,shift=1,drop_remainder=True)
training_data=training_data.flat_map(lambda x: x.batch(window_size+1))
training_data=training_data.shuffle(shuffle_buffer_size)
training_data=training_data.map(lambda x:(x[:-1],x[1:]))
training_data=training_data.batch(batch_size).prefetch(1)


In [8]:
model=Sequential([Lambda(lambda x:tf.expand_dims(x,axis=-1)),
                  Bidirectional(LSTM(32,return_sequences=True)),
                  Bidirectional(LSTM(64,return_sequences=True)),
                  Dense(10,activation='relu'),
                  Dense(64,activation='relu'),
                  Dense(vocab_size+1,activation='softmax')])


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(training_data,epochs=5,verbose=1,batch_size=1024,validation_data=val_data)

In [ ]:
text=["How are yo"]
text=tokenizer.texts_to_sequences(text)
text=np.array(text)


In [ ]:
y=model.predict_classes(text)